In [1]:
import os
import glob
import numpy as np
import pandas as pd
import gc
import time
PATH_DATASET = "/kaggle/input/icecube-neutrinos-in-deep-ice"
DATA_DIR=PATH_DATASET

Starting with @jirkborovec's excellent notebook, we will modify the direction computation to take into account the time of the sensor detects.

A separate discussion topic (TBD) will give the derivation of the formula, which is as follows.  It is called the Line-fit method.
We assume that the particle is following the path $pos = r + v * t$ where $v$ is the velocity and $r$ is the reference position at time 0.

Let $\langle x \rangle$ be the average value of the quantity $x$, and 
    $ti$=vector of times of our measurements, and $ri$=vector of sensor positions (x,y,z)
    
Then 
    
$$v_{est} = \frac{ \langle ri * ti \rangle -  \langle r \rangle * \langle t \rangle}{\langle t^2 \rangle - \langle t \rangle ^2}$$
    
where $v_{est}$ is the estimated velocity vector of the charged particle, which we assume has the same
    trajectory as the neutrino that generated it.

References:  
1. Mirco Hunnefield Masters Thesis, *Online Reconstruction of Muon-Neutrino Events in IceCube using Deep Learning Techniques*
2. Kai Schatto, PhD Thesis, *Stacked searches for high-energy
neutrinos from blazars with IceCube*


In [2]:
geometry = pd.read_csv(os.path.join(DATA_DIR, "sensor_geometry.csv"))
geometry.set_index("sensor_id", inplace=True)
geometry = geometry.apply(np.float32)
geometry.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5160 entries, 0 to 5159
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       5160 non-null   float32
 1   y       5160 non-null   float32
 2   z       5160 non-null   float32
dtypes: float32(3)
memory usage: 100.8 KB


In [3]:
#! pip install -q scikit-spatial --no-index -f /kaggle/input/icecube-neutrino-eda-3d-interactive-viewer/frozen-packages/

In [4]:
import math
import warnings
warnings.filterwarnings('ignore')

def cartesian_to_sphere(x, y, z):
    # https://en.wikipedia.org/wiki/Spherical_coordinate_system
    x2y2 = x**2 + y**2
    r = math.sqrt(x2y2 + z**2)
    azimuth = math.acos(x / math.sqrt(x2y2)) * np.sign(y)
    zenith = math.acos(z / r)
    return azimuth, zenith

def adjust_sphere(azimuth, zenith):
    if zenith < 0:
        zenith += math.pi
        azimuth += math.pi
    if azimuth < 0:
        azimuth += math.pi * 2
    azimuth = azimuth % (2 * math.pi)
    return azimuth, zenith

def sphere_to_cartesian(azimuth, zenith):
    # see: https://stackoverflow.com/a/10868220/4521646
    x = math.cos(azimuth) * math.sin(zenith)
    y = math.sin(azimuth) * math.sin(zenith)
    z = math.cos(zenith)
    return x, y, z

**Here is our function implementing the above Line-fit method**

In [5]:
def compute_direction(r, t):
    """compute Line-fit using r vector and time vectors
    return v_est and r_est """
    def avg(p):
        if len(p.shape)==1:
            p=p.reshape(-1,1)
        return np.mean(p, axis=0)
    #r + v*t is path of particle
    q = avg(t**2)-avg(t)**2
    v_est = (avg(r*t) - avg(r)*avg(t))/q
    r_est = avg(r) - v_est*avg(t)
    #v_est is the direction of travel
    return v_est, r_est


## Prepare submission¶

An example submission with the correct columns and properly ordered event IDs. The sample submission is provided in the parquet format so it can be read quickly but your final submission must be a csv.

In [6]:
ssub = pd.read_parquet(os.path.join(PATH_DATASET, "sample_submission.parquet"))
ssub.set_index("event_id", inplace=True)
display(ssub.head())

,azimuth,zenith
event_id,,
2092,1,1
7344,1,1
9482,1,1


In [7]:
ssub = ssub.apply(np.float32)
display(ssub.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 2092 to 9482
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   azimuth  3 non-null      float32
 1   zenith   3 non-null      float32
dtypes: float32(2)
memory usage: 48.0+ bytes


None

### Fitting to all data

In [8]:
ls = glob.glob(os.path.join(PATH_DATASET, "test", "*.parquet"))

for batch_file in ls:
    print(f"processing: {batch_file}")
    df = pd.read_parquet(batch_file)
    # del df['time'], df['charge']
    # display(df.head())
    gc.collect()
    
    for eid, dfg in df.groupby("event_id"):
        dfg = dfg[~dfg['auxiliary']]
        dfg = dfg.merge(geometry, left_on="sensor_id", right_index=True)
        # TODO: for memory reason subsample point cloud
        if len(dfg) > 800:
            dfg = dfg.sort_values('charge', ascending=False)[:800]
        # display(dfg)
        #These are the sensor position coordinates
        xyz = dfg[['x','y','z']].values
        #These are the times of detection
        ti = dfg[['time']].values
        v_est,r_est = compute_direction(xyz, ti)
        #we need to switch direction because we want the direction the neutrino came from, but
        #v_est is the direction it is traveling
        v_est = -v_est
        azimuth_, zenith_ = cartesian_to_sphere(*v_est)
        azimuth_, zenith_ = adjust_sphere(azimuth_, zenith_)
        ssub.at[eid, "azimuth"] = azimuth_
        ssub.at[eid, "zenith"] = zenith_
        if len(df) < 1e5:
            print(f"Estimation event {eid} with azimuth={azimuth_} & zenith={zenith_}")

    del df, dfg
    gc.collect()
    time.sleep(1)

processing: /kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_661.parquet
Estimation event 2092 with azimuth=3.8157139100990816 & zenith=2.325573031281843
Estimation event 7344 with azimuth=nan & zenith=3.141592653589793
Estimation event 9482 with azimuth=4.345080034675934 & zenith=1.4038316968516735


### Finalize submission

In [9]:
ssub.fillna(0).to_csv('submission.csv', index=True)

!head submission.csv

event_id,azimuth,zenith
2092,3.815714,2.325573
7344,0.0,3.1415927
9482,4.34508,1.4038317
